In [1]:
from copy import deepcopy
from collections import defaultdict
import random
import numpy as np
import time 
import json
from sklearn.linear_model import LinearRegression

## Reading and initializing functions

In [2]:
def get_sudokus(filename, sudokusize=9):
    '''
    get_states is to read multiple sudokus in the format:
    ..8..4..5...64..5 etc.
    returns a list of sudoku board states as clauses in a dict
    '''
    sudokus = []
    with open(filename) as sudoku_states:
        for i, state in enumerate(sudoku_states):
            sudoku = {}
            pointers = {}
            row = 1
            index = 0
            for j, content in enumerate(state):
                if j%sudokusize == 0 and j > 0:
                    row+=1
                try:
                    int(content)
                    literal = int(str(row) + str(j%sudokusize+1) + content)
                    sudoku[index] = [literal]
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    index += 1
                except:
                    # is not convertible to int
                    continue
            sudokus.append((sudoku, pointers))
    return sudokus

def get_constraints_from_dimacs(filename, prevConstraints={}, prevPointers={}):
    '''
    Convert dimacs file to a dictionary of constraints
    and pointers. Allows merging with previous pointers and constraints.
    '''
    constraints = prevConstraints # dictionary with constraints as: clausenumber->[literals]
    pointers = prevPointers # dictionary with pointers to clause numbers for literals as: literals->[clausenumbers]
    initial_length = len(constraints)
    with open(filename) as f:
        for i, clause in enumerate(f):
            index = i + initial_length
            if clause[0] == 'c' or clause[0] == 'p':
                # skip comments etc.
                continue
            constraints[index]= []
            for literal in clause.split():
                literal = int(literal)
                if literal != 0:
                    constraints[index].append(literal)
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    
    return constraints, pointers

    

## DP helper functions for constraints updates

In [3]:
def assign_literal(constraints, pointers, literal, assignments, random=False):
    '''
    assign the given literal to be True.
    assign_literal is aimed to be one recursion stack, so deepcopies are made
    in order to properly backtrack
    '''
    constraintsCopied = deepcopy(constraints)
    pointersCopied = deepcopy(pointers)
    assignmentsCopied = deepcopy(assignments)
    assignmentsUpdated = update_assigments(assignmentsCopied, literal)
    constraintsUpdated, pointersUpdated = update_constraints(constraintsCopied, pointersCopied, literal)
    if random:
        extract_features(literal, constraints, pointers, assignments)
    return constraintsUpdated, pointersUpdated, assignmentsUpdated
     
def update_constraints(constraints, pointers, literal):
    '''
    literal: e.g. 112 or 112'
    Updates constraints and pointers by removing all clauses 
    with the given literal, because the clause becomes true.
    Remove counter-literal out of all clauses.
    '''
    ## Find clauses with literal and remove them
    
    for lit in literal:
        clauses=list(pointers[lit])
        for clause in clauses:
            for l in constraints[clause]:
                pointers[l].remove(clause)
                if len(pointers[l])==0 and l!=lit:
                    del pointers[l]
            del constraints[clause]
        del pointers[lit]
    
    ## Find clauses with counter-literal and remove its counter-literal
    
    for lit in literal:
        counterLiteral = -lit
        if counterLiteral in pointers.keys():
            counters=list(pointers[counterLiteral])
            for clause in counters:
                constraints[clause].remove(counterLiteral)
            del pointers[counterLiteral]
        else:
            continue
   
    return constraints, pointers

def update_assigments(assignments, literal):
    '''
    Keep track of assigned literals.
    Needed in order to return satisfiable solution of literals.
    '''
    for lit in literal:
        if lit in assignments:
            print("already assigned", lit, "(in this stack), returning...")
#             return
        else:
            assignments.append(lit)
    return assignments

def check_tautologies(constraints, pointers):
    '''
    Check for clauses with a tautology and remove
    them.
    '''
    foundOne = False
    for clause in constraints.keys():
        literals = constraints[clause]
        counterLiterals = [-x for x in literals]
        for i in literals:
            for j in counterLiterals:
                if i == j:
                    del constraints[clause]
                    pointers[i].remove(clause)
                    pointers[-i].remove(clause)
                    foundOne = True
                    break
    if foundOne:
        print("Found tautologies and removed corresponding clauses")
    else:
        print("No tautologies found")
                  
    return constraints, pointers  

### Extract features to use as regressor datapoints

In [4]:
def extract_features(literals, constraints, pointers, assignments, logging=True):
    
    literal = next(iter(literals))
    ## Look one step ahead
    constraintsNew, pointersNew, assignmentsNew = assign_literal(constraints, pointers, literals, assignments)
    
    ## Ratio of the polarity of this literal before assignment
    positive = len(pointers[literal])
    negative = len(pointers[-literal])
    total = positive+negative
    if total != 0:
        ratio = positive/(positive+negative)
    else:
        ratio = 0
    
    # Clauses that were resolved with this assignment
    clausesResolved = len(constraints) - len(constraintsNew)
    
    # Clauses that became empty with this assignment
    emptyClauses = 0
    for clause in constraintsNew.keys():
        if not constraintsNew[clause]:
            emptyClauses += 1
    
    ## Average clause length with given literal before assignment
    clauses = pointers[literal]
    lengths = [len(constraints[c]) for c in clauses]
    if len(lengths) != 0:
        mean = np.mean(lengths) 
    else:
        mean = 0
    
    if logging:
        global features
        features[literal] = {}
        features[literal]["posneg_ratio"] = ratio
        features[literal]["c_res"] = clausesResolved
        features[literal]["empty_c"] = emptyClauses  
        features[literal]["c_len"] = mean
        features[literal]["SAT"] = 0
    else:
        return [ratio, clausesResolved, emptyClauses, mean]
    

### Train linear regressor

In [5]:
def train(file='data.json'):
    X = []
    y = []
    with open(file) as f:  
        data = json.load(f)
        for d in data:
            row = list(d.values())
            X.append(row[:-1])
            y.append(row[-1])
    global regressor
    regressor = LinearRegression().fit(np.array(X), np.array(y))

## DP cases

In [6]:
def empty_clause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
        
    return False

def unit_clause(constraints): 
    units=[]
    negUnit=[]
    for clause in constraints.keys():
        if len(constraints[clause])==1:
            literal = constraints[clause][0]
            if literal>0:
                units.append(literal)
            else:
                negUnit.append(literal)
    if len(units)>0:
        return list(set(units))
    else:
        return list(set(negUnit))

def unit_propagate(constraints, pointers, assignments):
    '''
    Automatically assign the literal of all unit clauses
    until there are no unit clauses left.
    '''
    literals = unit_clause(constraints)
    if len(literals)>0:
        status=True
        constraints, pointers, assignments = assign_literal(constraints, pointers, literals, assignments)    
    else:
        status=False
    
    return constraints, pointers, assignments,status

def pure_literal(pointers):
    literals=list(pointers.keys())
    for literal in literals:
        if -literal not in literals:
            return [literal],True
        
    return None,False

def random_literal(pointers):
    literals=list(pointers.keys())  
    try:
        literal = random.choice(literals) 
    except:
        return []
    
    return [literal]

### Branch split heuristics

In [13]:
def linreg(constraints, pointers, assignments, b=5): 
    '''
    Linreg performs a prediction on a batch of literals and picks the highest scoring literal.
    '''
    if(len(pointers.keys()) < b):
        b = len(pointers.keys())
    if len(pointers.keys())==0:
        return []
    batch = random.sample(list(pointers.keys()), k=b)
    X = np.array([extract_features([literal], constraints, pointers, assignments, logging=False) for literal in batch])
    scores = regressor.predict(X) # globally available regressor
    literal = batch[np.argmax(scores)]
    return [literal]
    
#Mom's heuristic    
def mom(constraints,pointers,k):
    smallestClasses=[]
    minClass=np.inf
    for clause in constraints.keys():
        if len(constraints[clause])<minClass:
            minClass=len(constraints[clause])
            smallestClasses=[]
            smallestClasses.append(clause)
        elif len(constraints[clause])==minClass:
            smallestClasses.append(clause)
    value=0
    maxLiteral=None
    haveSeen=[]
    
    for literal in pointers.keys():
        if literal not in haveSeen and -literal not in haveSeen:
            X=len([x for x in pointers[literal] if x in smallestClasses])
            X_=len([x for x in pointers[-literal] if x in smallestClasses])
            momsValue=(X+X_)*2^k+(X+X_)
            haveSeen.append(literal)
            if momsValue>value:
                value=momsValue
                maxLiteral=literal
    if maxLiteral:
        return [maxLiteral]
    else:
        return []
    
#jeroslow wang heuristic
def jeroSloWang(constraints,pointers):
    j={}
    maxValue=0
    maxLiteral=None
    if len(pointers.keys())==0:
        return []
    for literal in pointers.keys():
        for clause in pointers[literal]:
            if literal not in j.keys():
                j[literal]=0
            j[literal]+=(2**(-len(constraints[clause])))
       
        if j[literal]>maxValue:
            maxValue=j[literal]
            maxLiteral=literal
            
    return [maxLiteral]
 

## DP main function

In [8]:
def DP(constraints, pointers, assignments,split): 
    '''
    DP main recursive function. Always returns satisfiable
    assignment for all constraints if there is one.
    '''
    
#     if len(constraints.keys())==0:
#         return assignments
   
#     if empty_clause(constraints):
#         return [] 
    
    status=True
    while status:
        constraints, pointers, assignments,status = unit_propagate(constraints, pointers, assignments)
        
    status=True
    while status:
        literal,status = pure_literal(pointers)
        if status:
            constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments,random=False)
    
    if len(constraints.keys())==0:
        return assignments
   
    if empty_clause(constraints):
        return [] 
    
    global assignmentCount
    assignmentCount += 1
    if split=="random":
        literal = random_literal(pointers)
        if len(literal)==0:
            return DP(constraints, pointers, assignments, split)
    if split=="jerow":
        literal = jeroSloWang(constraints,pointers)
        if len(literal)==0:
            return DP(constraints, pointers, assignments, split)
    if split=="moms":
        literal = mom(constraints,pointers,3)
        if len(literal)==0:
            return DP(constraints, pointers, assignments, split)
    if split=="linreg":
        literal = linreg(constraints, pointers, assignments, 5)
        if len(literal)==0:
            return DP(constraints, pointers, assignments, split)
        
    constraintsUpdated, pointersUpdated, assignmentsUpdated = assign_literal(constraints, pointers, literal, assignments, random=True)
    assignmentsFinal = DP(constraintsUpdated, pointersUpdated, assignmentsUpdated, split)
    if len(assignmentsFinal)>0:
        return assignmentsFinal
    else:
        constraints, pointers, assignments = assign_literal(constraints, pointers, [literal[0]*-1], assignments, random=True)
        return DP(constraints, pointers, assignments,split)

### Script

In [14]:
datapoints = []
# train_sudokus = [x for x in range(0,1000) if x%2 == 0]
test_sudokus = [x for x in range(0,1000) if x%2 == 1]

print("Training linear regressor...")
train('data.json')

run_statistics = {}

sudokus1000 = get_sudokus('1000 sudokus.txt')
heuristics = ["random", "moms", "jerow", "linreg"]
# heuristics = ["random"]
    
## Run DP
runs = 2
for run in range(0, runs):
    run_statistics[run] = {}
    for i in test_sudokus:
        run_statistics[run][i] = {}
        for h in heuristics:
            run_statistics[run][i][h] = {}
            ## Get board and rules
            
            print("Reading and analyzing...")
            constraints, pointers = sudokus1000[i]
            run_statistics[run][i][h]["beginAssigments"] = len(pointers)
            constraints, pointers = get_constraints_from_dimacs('sudoku-rules.txt', constraints, pointers)
            constraints, pointers = check_tautologies(constraints, pointers)

            print("Starting DP algorithm with "+ h + " splits...")
            assignmentCount = 0
            global features
            features = {}
            startTime = time.time()
            assignments = DP(constraints, pointers, [], split=h)
            endTime = time.time()
            if assignments:
                print("Solution found")
                print("Time to complete:", "%2f seconds"%(endTime-startTime))
                boardPositions = [a for a in assignments if a > 0]
                if len(boardPositions) > 81:
                    print("too many assignments for sudoku solution:", len(boardPositions))
                elif len(boardPositions) == 81:
                    print(np.reshape([int(str(pos)[2]) for pos in sorted(boardPositions)], (9,9)))
                print("Branch split choices made:", assignmentCount)
                run_statistics[run][i][h]["assignmentCount"] = assignmentCount
                run_statistics[run][i][h]["runTime"] = (endTime-startTime)
                for a in assignments:
                    if a in features.keys():
                        features[a]["SAT"] = 1
                for f in features.keys():
                    datapoints.append(features[f])
                if h == "linreg":
                    run_statistics[run][i][h]["batchSize"] = 3
            else:
                print("No solution found")
                
# if datapoints:                
#     with open('data.json', 'w') as outfile:
#         json.dump(datapoints, outfile)

with open('run_statistics2.json', 'w') as f:
    json.dump(run_statistics, f)

Training linear regressor...
Reading and analyzing...
No tautologies found
Starting DP algorithm with random splits...
Solution found
Time to complete: 0.393360 seconds
[[2 4 9 1 8 6 5 7 3]
 [7 3 5 9 4 2 1 8 6]
 [1 6 8 3 7 5 4 2 9]
 [5 1 2 6 9 7 3 4 8]
 [9 7 6 8 3 4 2 5 1]
 [4 8 3 2 5 1 9 6 7]
 [6 9 4 7 2 3 8 1 5]
 [3 2 7 5 1 8 6 9 4]
 [8 5 1 4 6 9 7 3 2]]
Branch split choices made: 0
Reading and analyzing...
No tautologies found
Starting DP algorithm with moms splits...
Solution found
Time to complete: 0.765635 seconds
[[2 4 9 1 8 6 5 7 3]
 [7 3 5 9 4 2 1 8 6]
 [1 6 8 3 7 5 4 2 9]
 [5 1 2 6 9 7 3 4 8]
 [9 7 6 8 3 4 2 5 1]
 [4 8 3 2 5 1 9 6 7]
 [6 9 4 7 2 3 8 1 5]
 [3 2 7 5 1 8 6 9 4]
 [8 5 1 4 6 9 7 3 2]]
Branch split choices made: 0
Reading and analyzing...
No tautologies found
Starting DP algorithm with jerow splits...
Solution found
Time to complete: 1.145051 seconds
[[2 4 9 1 8 6 5 7 3]
 [7 3 5 9 4 2 1 8 6]
 [1 6 8 3 7 5 4 2 9]
 [5 1 2 6 9 7 3 4 8]
 [9 7 6 8 3 4 2 5 1]
 [4 8 3 2 5

Solution found
Time to complete: 1.183088 seconds
[[7 1 5 9 8 6 4 2 3]
 [6 8 9 2 4 3 7 1 5]
 [3 2 4 1 7 5 9 6 8]
 [9 6 3 4 1 8 5 7 2]
 [8 4 1 7 5 2 6 3 9]
 [2 5 7 6 3 9 1 8 4]
 [1 3 2 5 6 4 8 9 7]
 [5 9 6 8 2 7 3 4 1]
 [4 7 8 3 9 1 2 5 6]]
Branch split choices made: 0
Reading and analyzing...
No tautologies found
Starting DP algorithm with random splits...
Solution found
Time to complete: 0.437397 seconds
[[5 8 2 9 4 6 7 3 1]
 [9 3 4 1 8 7 6 2 5]
 [6 7 1 2 3 5 8 4 9]
 [1 6 8 4 9 2 3 5 7]
 [3 4 5 7 6 8 9 1 2]
 [7 2 9 5 1 3 4 8 6]
 [4 5 6 8 7 1 2 9 3]
 [2 9 3 6 5 4 1 7 8]
 [8 1 7 3 2 9 5 6 4]]
Branch split choices made: 3
Reading and analyzing...
No tautologies found
Starting DP algorithm with moms splits...
Solution found
Time to complete: 2.814545 seconds
[[5 8 2 9 4 6 7 3 1]
 [9 3 4 1 8 7 6 2 5]
 [6 7 1 2 3 5 8 4 9]
 [1 6 8 4 9 2 3 5 7]
 [3 4 5 7 6 8 9 1 2]
 [7 2 9 5 1 3 4 8 6]
 [4 5 6 8 7 1 2 9 3]
 [2 9 3 6 5 4 1 7 8]
 [8 1 7 3 2 9 5 6 4]]
Branch split choices made: 14
Reading and an

Solution found
Time to complete: 0.929858 seconds
[[4 1 6 5 7 9 8 2 3]
 [9 5 7 8 2 3 4 6 1]
 [3 8 2 6 4 1 7 9 5]
 [1 7 4 3 9 5 6 8 2]
 [5 2 9 4 8 6 3 1 7]
 [6 3 8 7 1 2 5 4 9]
 [2 4 3 1 6 7 9 5 8]
 [8 9 5 2 3 4 1 7 6]
 [7 6 1 9 5 8 2 3 4]]
Branch split choices made: 0
Reading and analyzing...
No tautologies found
Starting DP algorithm with linreg splits...
Solution found
Time to complete: 1.222111 seconds
[[4 1 6 5 7 9 8 2 3]
 [9 5 7 8 2 3 4 6 1]
 [3 8 2 6 4 1 7 9 5]
 [1 7 4 3 9 5 6 8 2]
 [5 2 9 4 8 6 3 1 7]
 [6 3 8 7 1 2 5 4 9]
 [2 4 3 1 6 7 9 5 8]
 [8 9 5 2 3 4 1 7 6]
 [7 6 1 9 5 8 2 3 4]]
Branch split choices made: 0
Reading and analyzing...
No tautologies found
Starting DP algorithm with random splits...
Solution found
Time to complete: 0.506448 seconds
[[8 7 9 3 4 6 1 5 2]
 [1 2 5 9 8 7 6 4 3]
 [6 3 4 2 1 5 8 7 9]
 [5 8 7 6 3 9 4 2 1]
 [2 4 3 1 7 8 5 9 6]
 [9 6 1 5 2 4 3 8 7]
 [7 5 2 4 6 1 9 3 8]
 [4 1 8 7 9 3 2 6 5]
 [3 9 6 8 5 2 7 1 4]]
Branch split choices made: 3
Reading and a

Solution found
Time to complete: 16.583067 seconds
[[1 3 7 8 9 2 4 5 6]
 [2 8 9 6 4 5 7 3 1]
 [6 5 4 1 7 3 9 2 8]
 [8 6 3 7 2 1 5 4 9]
 [7 2 1 9 5 4 6 8 3]
 [9 4 5 3 6 8 2 1 7]
 [3 7 2 5 8 9 1 6 4]
 [4 1 6 2 3 7 8 9 5]
 [5 9 8 4 1 6 3 7 2]]
Branch split choices made: 63
Reading and analyzing...
No tautologies found
Starting DP algorithm with jerow splits...
Solution found
Time to complete: 128.604754 seconds
[[1 3 7 8 9 2 4 5 6]
 [2 8 9 6 4 5 7 3 1]
 [6 5 4 1 7 3 9 2 8]
 [8 6 3 7 2 1 5 4 9]
 [7 2 1 9 5 4 6 8 3]
 [9 4 5 3 6 8 2 1 7]
 [3 7 2 5 8 9 1 6 4]
 [4 1 6 2 3 7 8 9 5]
 [5 9 8 4 1 6 3 7 2]]
Branch split choices made: 789
Reading and analyzing...
No tautologies found
Starting DP algorithm with linreg splits...
Solution found
Time to complete: 38.326337 seconds
[[1 3 7 8 9 2 4 5 6]
 [2 8 9 6 4 5 7 3 1]
 [6 5 4 1 7 3 9 2 8]
 [8 6 3 7 2 1 5 4 9]
 [7 2 1 9 5 4 6 8 3]
 [9 4 5 3 6 8 2 1 7]
 [3 7 2 5 8 9 1 6 4]
 [4 1 6 2 3 7 8 9 5]
 [5 9 8 4 1 6 3 7 2]]
Branch split choices made: 73
Readin

Solution found
Time to complete: 0.285314 seconds
[[5 2 1 6 8 3 4 9 7]
 [4 3 9 7 2 1 8 5 6]
 [6 8 7 5 4 9 2 1 3]
 [9 6 8 1 7 4 3 2 5]
 [7 1 4 3 5 2 9 6 8]
 [2 5 3 8 9 6 7 4 1]
 [1 7 2 9 3 5 6 8 4]
 [8 4 6 2 1 7 5 3 9]
 [3 9 5 4 6 8 1 7 2]]
Branch split choices made: 0
Reading and analyzing...
No tautologies found
Starting DP algorithm with moms splits...
Solution found
Time to complete: 0.545036 seconds
[[5 2 1 6 8 3 4 9 7]
 [4 3 9 7 2 1 8 5 6]
 [6 8 7 5 4 9 2 1 3]
 [9 6 8 1 7 4 3 2 5]
 [7 1 4 3 5 2 9 6 8]
 [2 5 3 8 9 6 7 4 1]
 [1 7 2 9 3 5 6 8 4]
 [8 4 6 2 1 7 5 3 9]
 [3 9 5 4 6 8 1 7 2]]
Branch split choices made: 0
Reading and analyzing...
No tautologies found
Starting DP algorithm with jerow splits...
Solution found
Time to complete: 1.039958 seconds
[[5 2 1 6 8 3 4 9 7]
 [4 3 9 7 2 1 8 5 6]
 [6 8 7 5 4 9 2 1 3]
 [9 6 8 1 7 4 3 2 5]
 [7 1 4 3 5 2 9 6 8]
 [2 5 3 8 9 6 7 4 1]
 [1 7 2 9 3 5 6 8 4]
 [8 4 6 2 1 7 5 3 9]
 [3 9 5 4 6 8 1 7 2]]
Branch split choices made: 0
Reading and anal

...

Solution found
Time to complete: 13.913309 seconds
[[9 4 1 5 2 6 3 7 8]
 [3 7 5 8 4 1 9 2 6]
 [2 6 8 7 9 3 4 5 1]
 [4 3 2 9 6 5 8 1 7]
 [6 8 9 2 1 7 5 3 4]
 [5 1 7 4 3 8 6 9 2]
 [7 2 6 3 8 9 1 4 5]
 [1 9 4 6 5 2 7 8 3]
 [8 5 3 1 7 4 2 6 9]]
Branch split choices made: 16
Reading and analyzing...
No tautologies found
Starting DP algorithm with random splits...
Solution found
Time to complete: 8.290174 seconds
[[6 1 5 3 7 2 8 4 9]
 [7 3 4 8 9 1 5 2 6]
 [9 8 2 4 6 5 7 3 1]
 [4 6 8 2 3 7 1 9 5]
 [1 5 9 6 8 4 2 7 3]
 [2 7 3 1 5 9 4 6 8]
 [8 9 7 5 2 6 3 1 4]
 [3 4 6 7 1 8 9 5 2]
 [5 2 1 9 4 3 6 8 7]]
Branch split choices made: 0
Reading and analyzing...
No tautologies found
Starting DP algorithm with moms splits...
Solution found
Time to complete: 1.425836 seconds
[[6 1 5 3 7 2 8 4 9]
 [7 3 4 8 9 1 5 2 6]
 [9 8 2 4 6 5 7 3 1]
 [4 6 8 2 3 7 1 9 5]
 [1 5 9 6 8 4 2 7 3]
 [2 7 3 1 5 9 4 6 8]
 [8 9 7 5 2 6 3 1 4]
 [3 4 6 7 1 8 9 5 2]
 [5 2 1 9 4 3 6 8 7]]
Branch split choices made: 0
Reading and a

Solution found
Time to complete: 12.406684 seconds
[[5 1 4 8 7 9 6 3 2]
 [2 6 3 4 5 1 8 9 7]
 [7 8 9 3 6 2 5 1 4]
 [9 7 6 5 2 8 3 4 1]
 [4 3 1 6 9 7 2 5 8]
 [8 5 2 1 3 4 7 6 9]
 [1 4 5 7 8 6 9 2 3]
 [6 2 8 9 4 3 1 7 5]
 [3 9 7 2 1 5 4 8 6]]
Branch split choices made: 68
Reading and analyzing...
No tautologies found
Starting DP algorithm with linreg splits...
Solution found
Time to complete: 6.547436 seconds
[[5 1 4 8 7 9 6 3 2]
 [2 6 3 4 5 1 8 9 7]
 [7 8 9 3 6 2 5 1 4]
 [9 7 6 5 2 8 3 4 1]
 [4 3 1 6 9 7 2 5 8]
 [8 5 2 1 3 4 7 6 9]
 [1 4 5 7 8 6 9 2 3]
 [6 2 8 9 4 3 1 7 5]
 [3 9 7 2 1 5 4 8 6]]
Branch split choices made: 9
Reading and analyzing...
No tautologies found
Starting DP algorithm with random splits...
Solution found
Time to complete: 1.618375 seconds
[[1 8 4 9 6 7 5 2 3]
 [7 6 9 5 2 3 4 1 8]
 [3 5 2 1 8 4 6 9 7]
 [8 7 6 4 5 9 2 3 1]
 [4 9 3 2 1 8 7 6 5]
 [5 2 1 7 3 6 9 8 4]
 [9 3 5 6 7 1 8 4 2]
 [2 4 8 3 9 5 1 7 6]
 [6 1 7 8 4 2 3 5 9]]
Branch split choices made: 5
Reading and

Solution found
Time to complete: 1.919308 seconds
[[8 5 4 1 2 6 9 3 7]
 [7 3 2 4 9 5 1 8 6]
 [9 1 6 7 8 3 4 5 2]
 [1 4 8 3 5 7 6 2 9]
 [2 9 5 8 6 1 7 4 3]
 [6 7 3 2 4 9 5 1 8]
 [3 2 9 6 1 4 8 7 5]
 [5 8 1 9 7 2 3 6 4]
 [4 6 7 5 3 8 2 9 1]]
Branch split choices made: 1
Reading and analyzing...
No tautologies found
Starting DP algorithm with jerow splits...
Solution found
Time to complete: 4.823791 seconds
[[8 5 4 1 2 6 9 3 7]
 [7 3 2 4 9 5 1 8 6]
 [9 1 6 7 8 3 4 5 2]
 [1 4 8 3 5 7 6 2 9]
 [2 9 5 8 6 1 7 4 3]
 [6 7 3 2 4 9 5 1 8]
 [3 2 9 6 1 4 8 7 5]
 [5 8 1 9 7 2 3 6 4]
 [4 6 7 5 3 8 2 9 1]]
Branch split choices made: 22
Reading and analyzing...
No tautologies found
Starting DP algorithm with linreg splits...
Solution found
Time to complete: 2.957382 seconds
[[8 5 4 1 2 6 9 3 7]
 [7 3 2 4 9 5 1 8 6]
 [9 1 6 7 8 3 4 5 2]
 [1 4 8 3 5 7 6 2 9]
 [2 9 5 8 6 1 7 4 3]
 [6 7 3 2 4 9 5 1 8]
 [3 2 9 6 1 4 8 7 5]
 [5 8 1 9 7 2 3 6 4]
 [4 6 7 5 3 8 2 9 1]]
Branch split choices made: 3
Reading and a

Solution found
Time to complete: 1.236749 seconds
[[1 3 2 8 7 4 6 9 5]
 [9 6 5 1 3 2 4 8 7]
 [7 8 4 9 5 6 1 3 2]
 [6 1 3 2 8 9 7 5 4]
 [2 4 9 5 1 7 8 6 3]
 [8 5 7 6 4 3 9 2 1]
 [5 7 6 4 2 8 3 1 9]
 [3 2 8 7 9 1 5 4 6]
 [4 9 1 3 6 5 2 7 8]]
Branch split choices made: 2
Reading and analyzing...
No tautologies found
Starting DP algorithm with moms splits...
Solution found
Time to complete: 3.543481 seconds
[[1 3 2 8 7 4 6 9 5]
 [9 6 5 1 3 2 4 8 7]
 [7 8 4 9 5 6 1 3 2]
 [6 1 3 2 8 9 7 5 4]
 [2 4 9 5 1 7 8 6 3]
 [8 5 7 6 4 3 9 2 1]
 [5 7 6 4 2 8 3 1 9]
 [3 2 8 7 9 1 5 4 6]
 [4 9 1 3 6 5 2 7 8]]
Branch split choices made: 8
Reading and analyzing...
No tautologies found
Starting DP algorithm with jerow splits...
Solution found
Time to complete: 4.951176 seconds
[[1 3 2 8 7 4 6 9 5]
 [9 6 5 1 3 2 4 8 7]
 [7 8 4 9 5 6 1 3 2]
 [6 1 3 2 8 9 7 5 4]
 [2 4 9 5 1 7 8 6 3]
 [8 5 7 6 4 3 9 2 1]
 [5 7 6 4 2 8 3 1 9]
 [3 2 8 7 9 1 5 4 6]
 [4 9 1 3 6 5 2 7 8]]
Branch split choices made: 27
Reading and ana

Solution found
Time to complete: 17.872168 seconds
[[7 4 6 8 9 2 3 1 5]
 [8 5 2 3 1 4 6 9 7]
 [1 3 9 6 7 5 4 2 8]
 [5 8 3 9 6 1 7 4 2]
 [2 7 4 5 8 3 9 6 1]
 [9 6 1 4 2 7 5 8 3]
 [3 2 8 7 4 6 1 5 9]
 [4 9 7 1 5 8 2 3 6]
 [6 1 5 2 3 9 8 7 4]]
Branch split choices made: 18
Reading and analyzing...
No tautologies found
Starting DP algorithm with random splits...
Solution found
Time to complete: 6.186346 seconds
[[1 6 4 2 7 8 9 5 3]
 [9 7 2 5 3 4 8 6 1]
 [3 5 8 1 9 6 4 7 2]
 [4 2 7 9 6 1 5 3 8]
 [8 3 6 7 5 2 1 9 4]
 [5 9 1 4 8 3 7 2 6]
 [2 8 9 6 1 5 3 4 7]
 [6 1 5 3 4 7 2 8 9]
 [7 4 3 8 2 9 6 1 5]]
Branch split choices made: 12
Reading and analyzing...
No tautologies found
Starting DP algorithm with moms splits...
Solution found
Time to complete: 48.551683 seconds
[[1 6 4 2 7 8 9 5 3]
 [9 7 2 5 3 4 8 6 1]
 [3 5 8 1 9 6 4 7 2]
 [4 2 7 9 6 1 5 3 8]
 [8 3 6 7 5 2 1 9 4]
 [5 9 1 4 8 3 7 2 6]
 [2 8 9 6 1 5 3 4 7]
 [6 1 5 3 4 7 2 8 9]
 [7 4 3 8 2 9 6 1 5]]
Branch split choices made: 77
Reading an

In [11]:
len(datapoints)

28894